In [1]:
import os

In [2]:
%pwd

'/home/aaronl/Downloads/NorthCarolina_CameroonChapter_AngusIssues/research'

In [3]:
os.chdir("../")

In [118]:
%pwd

'/home/aaronl/Downloads/NorthCarolina_CameroonChapter_AngusIssues'

In [217]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: float

In [219]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [220]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema
        

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = list(self.schema['TARGET_COLUMN'].keys())[0]
           
        )

        return model_evaluation_config


In [221]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [232]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):
        # Load and prepare test data
        test_data = pd.read_csv(self.config.test_data_path).sort_values(by='Date')
        test_data['Date'] = pd.to_datetime(test_data['Date'])
        test_data.set_index('Date', inplace=True)

        # Load the model
        model = joblib.load(self.config.model_path)

        # Prepare features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        # Make predictions
        start = 0  # Starting index
        end = len(test_x) - 1  # Ending index
        # Predict the target (without `start`)
        # Example: Using start and end for time series forecasting
        predicted_qualities = model.predict(start=start,end=end)
       

        # Evaluate metrics
        rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

        # Save metrics as a JSON file
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)




In [233]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    
    model_evaluation_config.save_results()
except Exception as e:
    raise e


[2024-08-12 00:31:54,713: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-12 00:31:54,716: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-12 00:31:54,718: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-12 00:31:54,719: INFO: common: created directory at: artifacts]
[2024-08-12 00:31:54,719: INFO: common: created directory at: artifacts/model_evaluation]
[2024-08-12 00:31:54,736: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
